In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import re
import nltk
import pandas as pd
# from urllib.parse import quote

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_3974/2944981425.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
df = pd.read_csv("../P3/text.csv")

In [6]:
df.head()

,Unnamed: 0,docs,category
0,0,sport pertains to any form of physical activit...,Sports
1,1,casa do pessoal do porto do lobito or simply k...,Sports
2,2,gombe united football club is a nigerian footb...,Sports
3,3,aibi international is a company operating in t...,Sports
4,4,an airboard is an inflatable bodyboard for the...,Sports


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

unigram_vectorizer = CountVectorizer(strip_accents="ascii")
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), strip_accents="ascii")
tfidf_vectorizer = TfidfVectorizer(strip_accents="ascii")

In [8]:
X = df["docs"]
y = df["category"]

In [10]:
unigram_matrix = unigram_vectorizer.fit_transform(X).toarray()
bigram_matrix = bigram_vectorizer.fit_transform(X).toarray()
tfidf_matrix = tfidf_vectorizer.fit_transform(X).toarray()

In [11]:
unigram_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]])

In [12]:
unique_words = [None for _ in range(len(unigram_vectorizer.vocabulary_))]
for word, idx in unigram_vectorizer.vocabulary_.items(): unique_words[idx] = word
uni_df = pd.DataFrame(unigram_matrix, columns=unique_words)
uni_df.head()

,10,100,100m,118,12,120,13,1300,14,15,...,yet,york,you,young,younger,youth,zealand,zigzag,zoomers,zoos
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
unique_bigrams = [None for _ in range(len(bigram_vectorizer.vocabulary_))]
for word, idx in bigram_vectorizer.vocabulary_.items(): unique_bigrams[idx] = word
bi_df = pd.DataFrame(bigram_matrix, columns=unique_bigrams).astype("float")
for idx, row in bi_df.iterrows():
    for bigram in unique_bigrams:
        prev = bigram.split(" ")[0]
        prev_count = uni_df[prev][idx]
        if prev_count != 0:
            row[bigram] /= prev_count
bi_df.head()

,10 and,10 climb,10 however,10 of,100 sitting,100m long,100m run,100m to,118 campus,12 swing,...,you can,young children,young men,younger children,youth and,youth through,zealand slovakia,zigzag balance,zoomers is,zoos and
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
labels = [None for _ in range(len(tfidf_vectorizer.vocabulary_))]
for word, idx in tfidf_vectorizer.vocabulary_.items(): labels[idx] = word
tfidf_df = pd.DataFrame(tfidf_matrix, columns=labels)
tfidf_df.head()

,10,100,100m,118,12,120,13,1300,14,15,...,yet,york,you,young,younger,youth,zealand,zigzag,zoomers,zoos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019264,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.041776,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.179287,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.182381,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
import numpy as np

eps = np.finfo(float).eps

def laplace_correction(data, var, Class, target, k):
    num = k
    den = len(data[data[Class] == target]) + len(np.unique(data[var]))
    return num / (den + eps)

def naive_bayes_classifier(data, Class, instance):
    max_prob, max_prob_target = -np.inf, None
    
    targets = list(np.unique(data[Class]))
    vars = [v for v in list(data.keys())[1:] if v != Class]
    
    for target in targets:
        p = len(data[data[Class] == target]) / (len(data) + eps)
        print("\np({}) = {:.5f}".format(target, p))
        
        for var in vars:
            num = len(data[var][data[Class] == target][data[var] == instance[var][0]])
            if num != 0:
                den = len(data[var][data[Class] == target])
                p_var = num/(den + eps)
            else:
                # Apply Laplace correction
                p_var = laplace_correction(data, var, Class, target, k=1)
            print("P({} = {} | {}) = {:.5f}".format(var, instance[var], target, p_var))
            p = p * p_var
        
        # print("P({} | {}) = {:.5f}".format(target, list(instance.values()), p))
        if p > max_prob:
            max_prob = p
            max_prob_target = target
    
    return max_prob, max_prob_target

In [85]:
new_doc = pd.Series(["sports is good for health"])

In [86]:
out = unigram_vectorizer.transform(new_doc).toarray()
uni_test = pd.DataFrame(out, columns=unigram_vectorizer.get_feature_names_out())

In [87]:
out = bigram_vectorizer.transform(new_doc).toarray()
bi_test = pd.DataFrame(out, columns=unique_bigrams)
for bigram in unique_bigrams:
    prev = bigram.split(" ")[0]
    prev_count = uni_test[prev][0]
    if prev_count != 0:
        row[bigram] /= prev_count

In [88]:
out = tfidf_vectorizer.transform(new_doc).toarray()
tfid_test = pd.DataFrame(out, columns=tfidf_vectorizer.get_feature_names_out())

In [89]:
naive_bayes_classifier(uni_df.assign(category=y), "category", uni_test)


p(Education) = 0.50000
P(100 = 0    0
Name: 100, dtype: int64 | Education) = 0.90000
P(100m = 0    0
Name: 100m, dtype: int64 | Education) = 1.00000
P(118 = 0    0
Name: 118, dtype: int64 | Education) = 0.90000
P(12 = 0    0
Name: 12, dtype: int64 | Education) = 1.00000
P(120 = 0    0
Name: 120, dtype: int64 | Education) = 0.90000
P(13 = 0    0
Name: 13, dtype: int64 | Education) = 1.00000
P(1300 = 0    0
Name: 1300, dtype: int64 | Education) = 1.00000
P(14 = 0    0
Name: 14, dtype: int64 | Education) = 1.00000
P(15 = 0    0
Name: 15, dtype: int64 | Education) = 0.90000
P(15th = 0    0
Name: 15th, dtype: int64 | Education) = 0.90000
P(16 = 0    0
Name: 16, dtype: int64 | Education) = 1.00000
P(165 = 0    0
Name: 165, dtype: int64 | Education) = 0.90000
P(1650s = 0    0
Name: 1650s, dtype: int64 | Education) = 0.90000
P(17th = 0    0
Name: 17th, dtype: int64 | Education) = 0.90000
P(18 = 0    0
Name: 18, dtype: int64 | Education) = 0.90000
P(1862 = 0    0
Name: 1862, dtype: int64 | Edu

P(2019 = 0    0
Name: 2019, dtype: int64 | Education) = 0.80000
P(2020 = 0    0
Name: 2020, dtype: int64 | Education) = 0.90000
P(2022 = 0    0
Name: 2022, dtype: int64 | Education) = 0.90000
P(23 = 0    0
Name: 23, dtype: int64 | Education) = 0.90000
P(24 = 0    0
Name: 24, dtype: int64 | Education) = 1.00000
P(25 = 0    0
Name: 25, dtype: int64 | Education) = 0.90000
P(26 = 0    0
Name: 26, dtype: int64 | Education) = 0.90000
P(2m = 0    0
Name: 2m, dtype: int64 | Education) = 1.00000
P(30 = 0    0
Name: 30, dtype: int64 | Education) = 0.90000
P(36 = 0    0
Name: 36, dtype: int64 | Education) = 0.90000
P(400m = 0    0
Name: 400m, dtype: int64 | Education) = 1.00000
P(49 = 0    0
Name: 49, dtype: int64 | Education) = 0.90000
P(50 = 0    0
Name: 50, dtype: int64 | Education) = 1.00000
P(500 = 0    0
Name: 500, dtype: int64 | Education) = 1.00000
P(513 = 0    0
Name: 513, dtype: int64 | Education) = 0.90000
P(570 = 0    0
Name: 570, dtype: int64 | Education) = 0.90000
P(593 = 0    0
Nam

(1.6231289272875135e-95, 'Sports')

In [58]:
naive_bayes_classifier(bi_df.assign(category=y), "category", bi_test)

(1.0352296571629177e-115, 'Sports')

In [59]:
naive_bayes_classifier(tfidf_df.assign(category=y), "category", tfid_test)

(3.407930056453156e-74, 'Sports')